In [7]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import json
import re
from tqdm import tqdm # 用於顯示進度條
import csv

# --- 1. 常數與提示詞定義 (來自您的 GRPO 腳本) ---

SYSTEM_PROMPT = """你是一名專門根據患者用藥與病史紀錄預測 ICD-10 編碼的醫學專家。  
請嚴格遵守以下作答格式：  

<reasoning>  
逐步推理：逐條說明病例中每個臨床資訊（症狀、診斷、檢查結果、既往病史、用藥等）可能對應的 ICD-10 編碼，並在最後總結出最合理的編碼集合。  
</reasoning>  
<answer>  
僅輸出最終的 ICD-10 編碼，使用英文逗號分隔，不要添加任何額外文字或解釋。  
</answer>  
"""

EXAMPLE_QUESTION = """
Sex: F
Service: MEDICINE
Allergies: Aspirin
Attending: ___
Chief Complaint: weakness, diarrhea
Major Surgical or Invasive Procedure: None

History of Present Illness:
Ms. ___ is a ___ year-old woman with PMH significant for chronic anemia, osteoporosis, hypertension, ataxia, and recent L5 fracture in the setting of recurrent falls who presents from home with fatigue and generalized weakness and diarrhea.  

The patient's recent history is notable for the follow:  
- On ___, she presented with 4 days of LBP s/p fall from standing at which time imaging revealed acute L5 fracture. She was evaluated by Spine team who recommended early mobilization, pain control, but no brace required. She was evaluated by ___, and discharged to ___.  
- She was discharged home with ___ on ___.  
- On ___, she again presented to ___ s/p fall from standing while trying to reach for a glass of water. She did have a occipital scalp hematoma, but imaging including ___, C-spine CT, and L hip X-ray were negative for acute process so patient was discharged home.  

She now represents with generalized fatigue and diarrhea. In the setting of opiates for her L5 fracture, the patient has had constipation (5 days with no BM) for which she took a "natural laxative" the evening prior to presentation. The patient had 2 bowel movements in the morning of presentation and one episode of incontinence with diarrhea while sleeping. In this setting, she felt very weak and called EMS and was brought to ___ ED.

What ICD-10 codes should be assigned? Please just tell me which codes in the end
"""
EXAMPLE_REASONING = """The patient has chronic anemia, documented and clinically relevant, which corresponds to D500.  
There is vitamin deficiency suggested, which fits E538.  
She also has visual impairment history, which maps to H548.  
Hypertension is a chronic condition, coded as I10.  
Diarrhea due to laxative use is coded as K521 (toxic gastroenteritis and colitis).  
Her recent fracture with underlying osteoporosis corresponds to M810.  
Ataxia is a chronic neurologic condition, coded as R270.  
There is adverse effect of drugs (laxatives/opioids), so T474X5A applies.  
Environmental factor of injury is Y92099 (unspecified place of occurrence).  
She also has aspirin allergy, captured by Z9181.  

Therefore, the most appropriate ICD-10 codes are:
</reasoning>
"""

EXAMPLE_ANSWER = "D500 E538 H548 I10 K521 M810 R270 T474X5A Y92099 Z9181"

In [8]:
def extract_icd_answer(text: str) -> str:
    """
    從模型的完整回覆或標準答案中提取 ICD-10 編碼。
    優先從 <answer> 標籤中提取,若無則從整個文本中提取。
    """
    # 使用正則表達式搜尋 <answer> 標籤，re.DOTALL 讓 . 可匹配換行符，re.IGNORECASE 忽略大小寫
    answer_match = re.search(r'<answer>(.*?)</answer>', text, re.DOTALL | re.IGNORECASE)
    # 如果找到 <answer> 標籤，則只在標籤內容中搜尋 ICD 編碼；否則在整個文本中搜尋
    search_text = answer_match.group(1) if answer_match else text
    
    # 匹配格式：字母 + 1-3位數字 + 可選的小數點和1-2位數字
    codes_list = re.findall(r'\b([A-Z]\d{1,6}(?:\.\d{1,2})?)\b', search_text, re.IGNORECASE)
    
    # 移除小數點、去重、排序、轉大寫，並過濾掉長度<=2的編碼
    cleaned_codes = sorted(set(
        code.replace('.', '').upper() 
        for code in codes_list 
        if len(code.replace('.', '')) > 2
    ))
    return " ".join(cleaned_codes)



def extract_xml_answer(text: str) -> str:
    """提取 <answer> 標籤內的內容"""
    answer = text.split("<answer>")[-1]
    answer = answer.split("</answer>")[0]
    answer = answer.strip()
    return answer


def is_text(text: str) -> bool:
    """判斷輸入字串是否主要為文字敘述（而非 ICD code 等短編碼）"""
    letters_and_spaces = sum(c.isalpha() or c.isspace() for c in text)
    ratio = letters_and_spaces / max(1, len(text))  
    return ratio > 0.5


def compute_f1(pred: str, true: str) -> float:
    """
    計算 F1 分數 (前三個字符匹配即算正確)
    
    Args:
        pred: 預測的 ICD 編碼字串,例如 "A010 B021"
        true: 真實的 ICD 編碼字串,例如 "A010 C032"
    
    Returns:
        F1 分數 (0.0 到 1.0)
    """
    # 將字串轉換為 set
    pred = pred.replace('.', '')
    true = true.replace('.', '')
    
    def parse_codes(text):
        if ',' in text:
            return set(code.strip().upper() for code in text.split(',') if code.strip())
        else:
            return set(code.strip().upper() for code in text.split() if code.strip())
    
    pred_set = parse_codes(pred)
    true_set = parse_codes(true)
    
    # 如果都是空集合,視為完全匹配
    if not pred_set and not true_set:
        return 1.0
    
    # 轉換為前三個字符的集合
    def get_prefix_set(codes):
        return set(code[:3] for code in codes if len(code) >= 3)
    
    pred_prefix = get_prefix_set(pred_set)
    true_prefix = get_prefix_set(true_set)
    
    # 計算 TP, FP, FN
    tp = len(pred_prefix & true_prefix)
    fp = len(pred_prefix - true_prefix)
    fn = len(true_prefix - pred_prefix)
    
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0.0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0.0
    
    if precision + recall == 0:
        return 0.0
    
    f1 = 2 * precision * recall / (precision + recall)
    print(f"✅ Precision: {precision:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}")
    return f1



def count_xml(text: str) -> float:
    """
    檢查 XML 標籤格式的完整性
    
    Returns:
        分數 (0.0 到 0.5),分數越高表示格式越完整
    """
    score = 0.0
    
    if text.count("<reasoning>\n") == 1:
        score += 0.125
    if text.count("\n</reasoning>\n") == 1:
        score += 0.125
    if text.count("\n<answer>\n") == 1:
        score += 0.125
        # 懲罰 </answer> 後有多餘內容
        extra_content = len(text.split("\n</answer>\n")[-1])
        score -= extra_content * 0.001
    if text.count("\n</answer>") == 1:
        score += 0.125
        # 懲罰 </answer> 後有多餘內容
        extra_content = len(text.split("\n</answer>")[-1]) - 1
        score -= max(0, extra_content) * 0.001
    
    return max(0.0, score)  # ✅ 確保不會是負數

In [9]:
# --- 3. 載入模型與 Tokenizer ---
model_name = "Qwen3-14B-sft_v1"
model_path = f"./models/{model_name}"

print(f"正在載入模型: {model_path} ...")
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    attn_implementation="flash_attention_2",
    trust_remote_code=True  # 關鍵修改
)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
print("模型載入完畢。")

# --- 4. 載入並處理資料 ---

print("正在載入 note_icd_data.jsonl ...")
chat_style_data = []
try:
    with open("note_icd_data.jsonl", "r", encoding="utf-8") as f:
        for i, line in enumerate(f):
            if i >= 450:
                print(f"已讀取前 {i} 筆資料 (上限 10 筆)。")
                break
            
            item = json.loads(line)  
            
            chat_prompt = [
                {"role": "system", "content": SYSTEM_PROMPT},
                {"role": "user", "content": EXAMPLE_QUESTION},
                {"role": "assistant", "content": f"<reasoning>{EXAMPLE_REASONING}</reasoning><answer>{EXAMPLE_ANSWER}</answer>"},
                {"role": "user", "content": item["question"]}
            ]
            
            ground_truth_answer= extract_icd_answer(item["answer"])
            
            chat_style_data.append({
                "prompt": chat_prompt,
                "answer": ground_truth_answer,
                "raw_question": item["question"]
            })
except FileNotFoundError:
    print("錯誤：找不到 'note_icd_data.jsonl' 檔案。")
    exit()

print(f"資料載入完畢，共 {len(chat_style_data)} 筆。")

正在載入模型: ./models/Qwen3-14B-sft_v1 ...


Loading checkpoint shards: 100%|██████████| 8/8 [00:04<00:00,  1.78it/s]

模型載入完畢。
正在載入 note_icd_data.jsonl ...
已讀取前 450 筆資料 (上限 10 筆)。
資料載入完畢，共 450 筆。


In [10]:
# --- 5. 執行 Baseline 推論並計算所有指標 ---
# 用於 Baseline Micro-F1
all_predictions_baseline = []
all_ground_truths_baseline = []
all_acc_baseline = []
all_strict_baseline = []
all_completion_lengths = []

# === 新增：初始化 CSV 檔案,寫入表頭 ===
csv_filename = f"test_{model_name}_1.csv"
with open(csv_filename, 'w', newline='', encoding='utf-8') as f:
    fieldnames = ['index', 'raw_question', 'accuracy_reward', 'completion_length', 'ground_truths_ans', 'predictions_ans', 'assistant_reply']
    writer = csv.DictWriter(f, fieldnames=fieldnames)
    writer.writeheader()

print(f"📁 CSV 檔案已初始化: {csv_filename}\n")

with torch.no_grad():
    for idx, item in enumerate(tqdm(chat_style_data, desc="[Baseline] 執行推論與評估"), 1):
        chat_prompt = item["prompt"]
        gt_answer = item["answer"]
        
        try:
            prompt_str = tokenizer.apply_chat_template(
                chat_prompt, 
                tokenize=False, 
                add_generation_prompt=True
            )
        except Exception as e:
            # 手動回退格式 (適用於 MedGemma)
            print(f"手動回退格式: {e}")
            prompt_str = (
                f"<start_of_turn>system\n{chat_prompt[0]['content']}<end_of_turn>\n"
                f"<start_of_turn>user\n{chat_prompt[1]['content']}<end_of_turn>\n"
                f"<start_of_turn>model\n{chat_prompt[2]['content']}<end_of_turn>\n"
                f"<start_of_turn>user\n{chat_prompt[3]['content']}<end_of_turn>\n"
                f"<start_of_turn>model\n"
            )
        
        inputs = tokenizer(
            prompt_str, 
            return_tensors="pt", 
            max_length=4096,
            truncation=True
        ).to(model.device)
        
        outputs = model.generate(
            **inputs,
            max_new_tokens=2000,
            do_sample=False,
            pad_token_id=tokenizer.eos_token_id
        )
        
        input_length = inputs.input_ids.shape[1]
        generated_tokens = outputs[0][input_length:]
        assistant_reply = tokenizer.decode(generated_tokens, skip_special_tokens=True)
        
        # === 印出完整資訊 ===
        print("\n" + "="*80)
        print(f"📝 樣本 #{idx}")
        print("="*80)
        print(f"\n🤖 模型完整回答:")
        print("-"*80)
        print(assistant_reply)
        print("-"*80)
        
        all_ground_truths_baseline.append(gt_answer)
        
        # 記錄 completion length (token 數)
        completion_length = len(generated_tokens)
        print(f"\n📊 完成的回覆長度 (token 數): {completion_length}")
        all_completion_lengths.append(completion_length)
        
        # 計算 Baseline metrics
        predicted_codes_baseline = extract_icd_answer(assistant_reply)
        print(f"🔍 提取的 ICD-10 編碼: {predicted_codes_baseline}")
        all_predictions_baseline.append(predicted_codes_baseline)

        print(f"\n🎯 Ground Truth Answer:")
        print(f"   {gt_answer}")
        
        acc_baseline = compute_f1(predicted_codes_baseline, gt_answer)
        print(f"✅ Baseline F1 分數: {acc_baseline:.4f}")
        all_acc_baseline.append(acc_baseline)
        
        
        with open(csv_filename, 'a', newline='', encoding='utf-8') as f:
            writer = csv.DictWriter(f, fieldnames=fieldnames)
            writer.writerow({
                'index': idx,
                'accuracy_reward': acc_baseline,
                'completion_length': completion_length,
                'ground_truths_ans': gt_answer,
                'predictions_ans': predicted_codes_baseline,
                'raw_question': chat_prompt, 
                'assistant_reply': assistant_reply if isinstance(assistant_reply, str) else json.dumps(assistant_reply)
            })
        
        print(f"💾 結果已儲存至 {csv_filename} (第 {idx} 筆)")
        print("="*80 + "\n")

print("\n" + "="*80)
print("🎉 推論完成!")
print("="*80)

# --- 6. 顯示最終統計 ---
if not all_predictions_baseline:
    print("沒有可評估的結果。")
else:
    print("\n" + "="*50)
    print("  Baseline (sft-4b-it) 最終統計")
    print("="*50)
    print(f"✅ 總共處理: {len(all_acc_baseline)} 筆資料")
    print(f"📊 平均 F1 分數: {sum(all_acc_baseline)/len(all_acc_baseline):.4f}")
    print(f"📏 平均回覆長度: {sum(all_completion_lengths)/len(all_completion_lengths):.1f} tokens")
    print(f"💾 結果已完整儲存至: {csv_filename}")
    print("="*50)

📁 CSV 檔案已初始化: test_Qwen3-14B-sft_v1_1.csv



[Baseline] 執行推論與評估:   0%|          | 1/450 [00:38<4:48:00, 38.49s/it]


📝 樣本 #1

🤖 模型完整回答:
--------------------------------------------------------------------------------
<think>

<reasoning>逐步推理：
1.  **Chief Complaint: weakness, diarrhea**：這是患者就診的主要原因，但需要根據病史和檢查結果進一步編碼。
2.  **PMH: chronic anemia**：病史中明確提到，且實驗室檢查顯示Hct 7.8/25.4，鐵蛋白249，血紅蛋白7.8，符合慢性貧血的診斷。編碼為 D500。
3.  **PMH: osteoporosis**：病史中明確提到，且有L5骨折史，是導致骨折的基礎疾病。編碼為 M810。
4.  **PMH: hypertension**：病史中明確提到。編碼為 I10。
5.  **PMH: ataxia**：病史中明確提到。編碼為 R270。
6.  **Recent L5 fracture in the setting of recurrent falls**：病史中明確提到，且有影像學證實。編碼為 S32011A (L5 fracture, initial encounter)。
7.  **Recurrent falls**：病史中明確提到多次跌倒，且導致骨折。編碼為 Y92099 (Unspecified place of occurrence) 作為環境因素。
8.  **Diarrhea**：患者主訴，且有「incontinence with diarrhea while sleeping」的描述。編碼為 K529 (Noninfective gastroenteritis and colitis, unspecified)。
9.  **Constipation (5 days with no BM) for which she took a "natural laxative" the evening prior to presentation. The patient had 2 bowel movements in the morning of presentation and one episode of incontine

[Baseline] 執行推論與評估:   0%|          | 2/450 [01:29<5:41:02, 45.68s/it]


📝 樣本 #2

🤖 模型完整回答:
--------------------------------------------------------------------------------
<think>

<reasoning>逐步推理：
1.  **Cardiogenic shock (Chief Complaint & HPI):** 患者因心源性休克入院，這是本次就診的主要原因。對應 ICD-10 編碼 I5022 (Acute on chronic systolic (congestive) heart failure with cardiogenic shock)。
2.  **Ischemic cardiomyopathy with EF 25% (PMHx):** 患者有缺血性心肌病史，且射血分數為25%，這表明心臟功能嚴重受損。對應 ICD-10 編碼 I5032 (Chronic systolic (congestive) heart failure, severe)。
3.  **3 vessel CAD s/p RCA stenting (PMHx):** 患者有三支血管冠狀動脈疾病史，並曾接受右冠狀動脈支架植入術。對應 ICD-10 編碼 I2510 (Atherosclerotic heart disease of native coronary artery without angina pectoris)。
4.  **DMII (PMHx):** 患者有第二型糖尿病史。對應 ICD-10 編碼 E119 (Type 2 diabetes mellitus without complications)。
5.  **HLD (PMHx):** 患者有高血脂史。對應 ICD-10 編碼 E785 (Hyperlipidemia, unspecified)。
6.  **Obesity (PMHx):** 患者有肥胖史。對應 ICD-10 編碼 E669 (Obesity, unspecified)。
7.  **HTN (PMHx):** 患者有高血壓史。對應 ICD-10 編碼 I10 (Essential (primary) hypertension)。
8.  **CKD (PMHx):** 患者有慢性腎臟病史。對應 

...


📝 樣本 #448

🤖 模型完整回答:
--------------------------------------------------------------------------------
<think>

<reasoning>Step-by-step reasoning:
1.  **Chief Complaint: Chest pain** - This is the primary reason for the encounter. The patient experienced acute onset of left-sided chest pressure, associated with nausea and dyspnea, which resolved after ~1hr. This is consistent with angina pectoris. Given the history of CAD and CABG, this is likely unstable angina. Therefore, **I20.0 (Unstable angina pectoris)** is appropriate.
2.  **History of Present Illness: History of CAD s/p CABG** - This indicates a significant past medical history. The patient has a history of Coronary Artery Disease and has undergone Coronary Artery Bypass Grafting. This is a chronic condition. **I25.10 (Atherosclerotic heart disease of native coronary artery without angina pectoris)** is used to capture the underlying CAD, as the current presentation is primarily angina.
3.  **History of Present Illness: Paroxys

[Baseline] 執行推論與評估: 100%|█████████▉| 449/450 [4:43:06<00:35, 35.09s/it]


📝 樣本 #449

🤖 模型完整回答:
--------------------------------------------------------------------------------
<think>

<reasoning>逐步推理：
1.  **Chief Complaint: Chest pain** - 患者主訴胸痛，且病史中明確提到「recurrent chest pain」和「persistent pericarditis-type pain」，因此 I20.9 (Angina pectoris, unspecified) 是一個合適的編碼。
2.  **History of Present Illness: hx of hypertension** - 患者有高血壓病史，對應 I10 (Essential (primary) hypertension)。
3.  **History of Present Illness: severe aortic stenosis, now 6 months s/p AVR with bioprosthetic valve** - 患者有嚴重主動脈瓣狹窄，並已接受生物瓣膜主動脈瓣置換術。由於是「s/p AVR」，表示是既往手術，因此 I36.1 (Aortic (valve) stenosis) 用於描述主動脈瓣狹窄，而 Z95.2 (Presence of aortic valve prosthesis) 用於表示生物瓣膜的存在。
4.  **History of Present Illness: post-op pericarditis** - 患者有術後心包炎病史，且目前仍有「persistent pericarditis-type pain」，因此 I36.0 (Pericarditis) 是一個合適的編碼。
5.  **History of Present Illness: known 40% mid-LAD occlusion** - 患者有已知的左前降支中段40%阻塞，這是一個冠狀動脈疾病，對應 I25.10 (Atherosclerotic heart disease of native coronary artery without angina pectoris)。雖然患者有胸

[Baseline] 執行推論與評估: 100%|██████████| 450/450 [4:43:33<00:00, 37.81s/it]


📝 樣本 #450

🤖 模型完整回答:
--------------------------------------------------------------------------------
Assistant response:
<reasoning>逐步推理：
1.  **Chief Complaint: right facial droop, slurred speech, hypertension, change in vision**：這些症狀提示急性中風。患者在ED的NIHSS評分為1，且頭部CT未顯示出血，因此最可能的診斷是缺血性中風。
2.  **History of Present Illness: stage IV lung cancer not on treatment**：這是患者的既往病史，應編碼為Z85.11 (Personal history of malignant neoplasm of bronchus and lung)。
3.  **DVTs on apixaban**：這是患者的既往病史，應編碼為Z85.828 (Personal history of other specified diseases of the circulatory system) 來表示深部靜脈血栓的個人史，並編碼Z79.01 (Long term (current) use of anticoagulants) 來表示長期使用抗凝劑。
4.  **Episode of hypertension to the 190s**：這是本次就診的急性問題，應編碼為I10 (Essential (primary) hypertension)。
5.  **slurred speech and right-sided tongue deviation as well as a question of right facial droop**：這些是中風的神經學症狀，支持缺血性中風的診斷。
6.  **Noncontrast head CT performed as discussed below. She was deemed not a candidate for TPA or endovascular intervention.**：這表明頭部